# ASTM E2022

In [1]:
import numpy as np
from scipy.interpolate import lagrange

import colour
from colour import CaseInsensitiveMapping, SpectralShape, tstack

np.set_printoptions(suppress=True)

CMFS = colour.STANDARD_OBSERVERS_CMFS.get(
        'CIE 1931 2 Degree Standard Observer')
A = colour.ILLUMINANTS_RELATIVE_SPDS['A'].clone().align(CMFS.shape)

_TRISTIMULUS_WEIGHTING_FACTORS_CACHE = None

_LAGRANGE_INTERPOLATING_COEFFICIENTS_CACHE = None


def lagrange_interpolating_coefficient(r, d=4):
    r_i = np.arange(d)
    L_n = []
    for j in range(len(r_i)):
        p = [(r - r_i[m]) / (r_i[j] - r_i[m])
             for m in range(len(r_i)) if m != j]
        L_n.append(reduce(lambda m, n: m * n, p))

    return L_n


def lagrange_interpolating_coefficients_ASTME2022(steps=10,
                                                  interval='intermediate'):
    global _LAGRANGE_INTERPOLATING_COEFFICIENTS_CACHE
    if _LAGRANGE_INTERPOLATING_COEFFICIENTS_CACHE is None:
        _LAGRANGE_INTERPOLATING_COEFFICIENTS_CACHE = CaseInsensitiveMapping()

    name_lica = ', '.join((str(steps), interval))
    if name_lica in _LAGRANGE_INTERPOLATING_COEFFICIENTS_CACHE:
        return _LAGRANGE_INTERPOLATING_COEFFICIENTS_CACHE[name_lica]

    r_n = np.linspace(1 / steps, 1 - (1 / steps), steps - 1)
    d = 3
    if interval.lower() == 'intermediate':
        r_n += 1
        d = 4

    lica = _LAGRANGE_INTERPOLATING_COEFFICIENTS_CACHE[name_lica] = (
        np.asarray([lagrange_interpolating_coefficient(r, d) for r in r_n]))

    return lica


def tristimulus_weighting_factors(cmfs, illuminant, shape):
    if cmfs.shape.steps != 1:
        raise RuntimeError('"{0}" shape "steps" must be 1!'.format(cmfs))

    if illuminant.shape.steps != 1:
        raise RuntimeError('"{0}" shape "steps" must be 1!'.format(illuminant))

    global _TRISTIMULUS_WEIGHTING_FACTORS_CACHE
    if _TRISTIMULUS_WEIGHTING_FACTORS_CACHE is None:
        _TRISTIMULUS_WEIGHTING_FACTORS_CACHE = CaseInsensitiveMapping()

    name_twf = ', '.join((cmfs.name, illuminant.name, str(shape)))
    if name_twf in _TRISTIMULUS_WEIGHTING_FACTORS_CACHE:
        return _TRISTIMULUS_WEIGHTING_FACTORS_CACHE[name_twf]

    c_intermediate = lagrange_interpolating_coefficients_ASTME2022(
            shape.steps, 'intermediate')
    c_boundaries = lagrange_interpolating_coefficients_ASTME2022(
            shape.steps, 'c_boundaries')

    x_bar, y_bar, z_bar = (cmfs.x_bar.values,
                           cmfs.y_bar.values,
                           cmfs.z_bar.values)
    S = illuminant.values

    k = 100 / np.sum(y_bar * S)

    W_x = k * S * x_bar
    W_y = k * S * y_bar
    W_z = k * S * z_bar

    W = _TRISTIMULUS_WEIGHTING_FACTORS_CACHE[name_twf] = (
        tstack((W_x, W_y, W_z)))

    return W


W = tristimulus_weighting_factors(CMFS, A, SpectralShape(360, 830, 10))


In [2]:
_TRISTIMULUS_WEIGHTING_FACTORS_CACHE.keys()

[u'CIE 1931 2 Degree Standard Observer, A, (360.0, 830.0, 10.0)']

In [3]:
# import pandas as pd
# pd.DataFrame(W, index=CMFS.clone().interpolate(SpectralShape(steps=5)).wavelengths)

In [4]:
class LagrangeInterpolator_ASTM2022(object):
    """
    Parameters
    ----------
    x : array_like
        Independent :math:`x` variable values corresponding with :math:`y`
        variable.
    y : array_like
        Dependent and already known :math:`y` variable values to
        interpolate.

    Methods
    -------
    __call__

    See Also
    --------
    LinearInterpolator

    Notes
    -----
    The minimum number :math:`k` of data points required along the
    interpolation axis is :math:`k=6`.

    References
    ----------

    Examples
    --------
    Interpolating a single numeric variable:

    >>> y = np.array([5.9200,
    ...               9.3700,
    ...               10.8135,
    ...               4.5100,
    ...               69.5900,
    ...               27.8007,
    ...               86.0500])
    >>> x = np.arange(len(y))
    >>> f = LagrangeInterpolator_ASTM2022(x, y)
    >>> f(0.5)  # doctest: +ELLIPSIS
    7.2185025...

    Interpolating an *array_like* variable:

    >>> f([0.25, 0.75])  # doctest: +ELLIPSIS
    array([ 6.7295161...,  7.8140625...])
    """

    def __init__(self, x=None, y=None):
        self.__x = None
        self.x = x
        self.__y = None
        self.y = y

        self.__validate_dimensions()

    @property
    def x(self):
        """
        Property for **self.__x** private attribute.

        Returns
        -------
        array_like
            self.__x
        """

        return self.__x

    @x.setter
    def x(self, value):
        """
        Setter for **self.__x** private attribute.

        Parameters
        ----------
        value : array_like
            Attribute value.
        """

        if value is not None:
            value = np.atleast_1d(value).astype(np.float_)

            assert value.ndim == 1, (
                '"x" independent variable must have exactly one dimension!')

        self.__x = value

    @property
    def y(self):
        """
        Property for **self.__y** private attribute.

        Returns
        -------
        array_like
            self.__y
        """

        return self.__y

    @y.setter
    def y(self, value):
        """
        Setter for **self.__y** private attribute.

        Parameters
        ----------
        value : array_like
            Attribute value.
        """

        if value is not None:
            value = np.atleast_1d(value).astype(np.float_)

            assert value.ndim == 1, (
                '"y" dependent variable must have exactly one dimension!')

            assert len(value) >= 3, (
                '"y" dependent variable values count must be in domain [3:]!')

        self.__y = value

    def __call__(self, x):
        """
        Evaluates the interpolating polynomial at given point(s).

        Parameters
        ----------
        x : numeric or array_like
            Point(s) to evaluate the interpolant at.

        Returns
        -------
        numeric or ndarray
            Interpolated value(s).
        """

        return self.__evaluate(x)

    def __evaluate(self, x):
        """
        Performs the interpolating polynomial evaluation at given point.

        Parameters
        ----------
        x : numeric
            Point to evaluate the interpolant at.

        Returns
        -------
        float
            Interpolated point values.
        """

        x = np.atleast_1d(x)

        self.__validate_dimensions()
        self.__validate_interpolation_range(x)

        y = []
        indexes = np.searchsorted(self.__x, x) - 1
        for i, ii in enumerate(indexes):
            if ii == 0:
                p = [ii, ii + 1, ii + 2]
            elif ii == len(self.__x) - 2:
                p = [ii - 1, ii, ii + 1]
            else:
                p = [ii - 1, ii, ii + 1, ii + 2]

            y.append(lagrange(self.__x[p], self.__y[p])(x[i]))

        return np.array(y)

    def __validate_dimensions(self):
        """
        Validates variables dimensions to be the same.
        """

        if len(self.__x) != len(self.__y):
            raise ValueError(
                ('"x" independent and "y" dependent variables have different '
                 'dimensions: "{0}", "{1}"').format(len(self.__x),
                                                    len(self.__y)))

    def __validate_interpolation_range(self, x):
        """
        Validates given point to be in interpolation range.
        """

        below_interpolation_range = x < self.__x[0]
        above_interpolation_range = x > self.__x[-1]

        if below_interpolation_range.any():
            raise ValueError('"{0}" is below interpolation range.'.format(x))

        if above_interpolation_range.any():
            raise ValueError('"{0}" is above interpolation range.'.format(x))


y = np.array([5.9200, 9.3700, 10.8135, 4.5100, 6.5900, 7.8007, 8.0500])
x = np.arange(len(y))
print(x)
n = 2.5
print(LagrangeInterpolator_ASTM2022(x, y)(n))
print(colour.LinearInterpolator(x, y)(n))
print(colour.SpragueInterpolator(x, y)(n))
print(colour.CubicSplineInterpolator(x, y)(n))

print('\n')
n = (0.1, 2.5, 3.5, 5.9)
print(LagrangeInterpolator_ASTM2022(x, y)(n))
print(colour.LinearInterpolator(x, y)(n))
print(colour.SpragueInterpolator(x, y)(n))
print(colour.CubicSplineInterpolator(x, y)(n))

print('\n')

y = np.array([5.9200, 9.3700, 10.8135])
x = np.arange(len(y))
print(LagrangeInterpolator_ASTM2022(x, y)((1, 1, 1, 1)))

[0 1 2 3 4 5 6]
[ 7.62196875]
7.66175
7.58080898437
7.49388148193


[ 6.3552925   7.62196875  5.0803625   8.068333  ]
[ 6.265    7.66175  5.55     8.02507]
[ 6.26844724  7.58080898  4.89025391  8.03687569]
[ 5.73319701  7.49388148  4.76535906  7.77059769]


[ 9.37  9.37  9.37  9.37]


In [5]:
S0 = colour.D_ILLUMINANTS_S_SPDS['S0']
x = S0.wavelengths
y = S0.values

n = 825

print(colour.LinearInterpolator(x, y)(n))
print(LagrangeInterpolator_ASTM2022(x, y)(n))
print(colour.CubicSplineInterpolator(x, y)(n))
print(colour.SpragueInterpolator(x, y)(n))

60.4
[ 60.725]
62.6293978601
60.997850628
